# Trio Mode

This notebook provides an overview of Jupyter's Trio mode. Make sure to run this notebook inside a Trio-mode kernel. 

## Using Trio APIs

In [5]:
import trio

The Trio kernel runs each cell as a Trio task on an event loop that is created at startup. This event loop continues running until the kernel terminates. This allows you to run asynchronous code that uses Trio APIs.

In [6]:
await trio.sleep(0)

You can also call Trio APIs that are not asynchronous:

In [7]:
trio.current_time()

20444.33086130232

Each cell runs until its underlying task completes. Unfortunately, the `%time` and `%%time` cell magics are not compatible with Trio mode, so we have to define our own timing mechanism.

In [8]:
from contextlib import contextmanager
from time import time

@contextmanager
def stopwatch():
    start = time()
    yield
    elapsed = time() - start
    print(f"Elapsed time: {elapsed:.2f}")

In [9]:
with stopwatch():
    await trio.sleep(2)

Elapsed time: 2.01


## Comparison to `%autoawait trio`

Jupyter also has a cell magic called `%autoawait trio` that lets you use Trio inside a notebook, but that feature has a few drawbacks:

* Doesn't work with synchronous Trio APIs (e.g. the `trio.current_time()` example above.)
* Doesn't allow background tasks.

The first point is inconvenient, but the second point is pretty much a deal-breaker. In `%autoawait trio` mode, Jupyter runs each cell inside of a new Trio event loop. This makes it impossible to spawn background tasks that keep running after a cell completes.

This is an especially poignant issue in Trio, because libraries may need to create background tasks as part of normal API functionality. For example, a WebSocket library may need a background task to read incoming network data. In `%autoawait trio` mode, the background task has to finish before a cell can complete, and therefore a connection created in one cell cannot be used in other cells!

For this reason, the Trio kernel does not use `%autoawait trio`–in fact, trying to execute it will throw an exception.

In [25]:
%autoawait trio

/Volumes/Case_Sensitive/third-party/ipykernel/ipykernel/trio_runner.py:20: UserWarning: Autoawait isn't allowed in Trio background loop mode.
  lambda _: warnings.warn("Autoawait isn't allowed in Trio "


## Background Tasks

The Trio kernel keeps the event loop open and runs each cell as a Trio task on this long-lived event loop, which solves the problem posed by `%autoawait trio`. Trio's principle of *structued concurrency* requires that all tasks are launched from within a nursery, and it's not possible to create a nursery in one cell and then use it in another cell. Therefore, the Trio kernel exposes a builtin called `GLOBAL_NURSERY` that can be used for spawning background tasks.

Let's look at the difference between creating a nursery and using the global nursery.

In [10]:
async def count_to(n):
    for i in range(n):
        await trio.sleep(1)
        now = time()
        print(f"{i} {now:.2f}")

In [11]:
with stopwatch():
    async with trio.open_nursery() as nursery:
        nursery.start_soon(count_to, 5)

0 1617236131.30
1 1617236132.30
2 1617236133.31
3 1617236134.31
4 1617236135.31
Elapsed time: 5.01


Notice how the cell takes 5 seconds to run. This is because it is waiting for the nursery to exit, and the nursery is waiting for the task to exit.

Now let's try an example where the same task is spawned in the global nursery. Since it doesn't make much sense for a background task to be printing out its results, we'll create a new function which updates a global variable once per second.

In [12]:
counter = 0
async def count():
    global counter
    while True:
        await trio.sleep(1)
        counter += 1

In [18]:
def report_count():
    global counter
    now = time()
    print(f"At time {now:.2f}, the count is {counter}")

We cam check if this background task is running by periodically checking the counter to see if it is growing.

In [19]:
report_count()

At time 1617236475.95, the count is 0


In [20]:
report_count()

At time 1617236500.82, the count is 0


Now let's start the count in the global nursery.

In [21]:
with stopwatch():
    GLOBAL_NURSERY.start_soon(count)

Elapsed time: 0.00


This cell returns immediately, because it is not waiting for `count()` function to return. (Indeed, `count()` never does return.) Instead, it is creating a new task in the background and then completing immediately.

Let's check on the counter.

In [22]:
report_count()

At time 1617236582.97, the count is 53


In [23]:
report_count()

At time 1617236588.42, the count is 58


This output tells us that the counter task is continuing to run in the background, and we can access its state from other cells in the notebook.

## Error Handling

The Trio kernel needs to handle exceptions carefully, lest it crash the solitary Trio event loop. Furthermore, a background task doesn't have a specific cell to write error messages to. We will demonstrate with the following task:

In [27]:
async def thrower():
    await trio.sleep(5)
    raise Exception("Exception raised in thrower()")

In [28]:
with stopwatch():
    GLOBAL_NURSERY.start_soon(thrower)

Elapsed time: 0.00


In [29]:
print("Hello")

Hello


ERROR:root:An exception occurred in a global nursery task.
Traceback (most recent call last):

  File "/var/folders/0p/5qcj97k95556jszmcwwy92jh0000gn/T/ipykernel_72166/358072194.py", line 3, in thrower
    raise Exception("Exception raised in thrower()")

Exception: Exception raised in thrower()



The exception is not displayed in the cell where the background task was created. In fact, that cell returns immediately, and then the user runs another cell. Instead, the exception is displayed *in the most recently executed cell*. This is partially a quirk of how Jupyter handles output, and also makes sense to display the exception where the user is currently looking, rather than where they were looking when they started the background task.

Finally, let's check on the background task again to ensure that the exception did not crash other background tasks:

In [30]:
report_count()

At time 1617237101.40, the count is 570


In [31]:
report_count()

At time 1617237106.86, the count is 575


Hooray! The background loop is still running.